In [1]:
import os
import datetime
from datetime import timedelta
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime

time_run = time.time()

In [2]:
xls = pd.ExcelFile("../Input file/Costing Examples_NEW.xlsx")
time_df = pd.read_excel(xls, 'Check')
holiday_df = xls.parse("Holiday")
holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

base_rate = 20
holiday_rate = 2.5
data= []
list_pay = []

rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
rate_data = {"Full Time": rate1, "Part Time": rate2, "Casual": rate3}

In [3]:
def date_to_str(date):
    date = str(date).split("-")
    return date[1] + "/" + date[2] 

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]

def cal_base_rate(_type, day, start_time, end_time, rate_data):
    
    shift = 2;      
    ord_rate = 0
    
    if date_to_str(start_date) in holiday_list:
        ord_rate = holiday_rate
    else:        
        if _type == "Full Time":
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "Part Time":
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or end_time > 1:
                shift = 2
            elif end_time > 19 or end_time < 1:
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
            
    return ord_rate

for row in time_df.iterrows():    
    _id = row[1]["Contact Name"] 
    
    start_period = row[1]['Process period start date'].date()
    end_period = row[1]['Process period end date'].date()
    
    start = row[1]["Start date"]
    end   = row[1]["End date"]
    
    start_time = int(start.hour)
    end_time   = int(end.hour)
        
    _type = row[1]["Employment Type"]
    
    start_date = start.date() 
    end_date   = end.date()
    
    delta_day = int((start_date- end_date).days)
    
    start_day  = start.strftime('%A')
    end_day    = end.strftime('%A')
    
    if start_date == end_date:
        ""
        delta_time = (end-start).total_seconds() / 3600.0
        rate = cal_base_rate(_type, start_day, start_time, end_time, rate_data)
        data.append((_id,start_period,end_period,start_date,start,end,start_day,end_day,delta_time, rate))
    else:
        if (end_time >=1) & (start_day in ["Friday","Saturday","Sunday"]):
            rate1 = cal_base_rate(_type, start_day, start_time, 24, rate_data)
            rate2 = cal_base_rate(_type, end_day, 0, end_time, rate_data)
            delta_time1 = 24 - start_time
            delta_time2 = end_time
            data.append((_id,start_period,end_period,start_date,start,end,start_day,end_day,delta_time1, rate1))
            data.append((_id,start_period,end_period,end_date,start_day,end_day,delta_time2, rate2))  
        else: 
            delta_time = (end-start).total_seconds() / 3600.0
            rate = cal_base_rate(_type, start_day, start_time, end_time, rate_data)
            data.append((_id,start_period,end_period,start_date,start,end,start_day,end_day,delta_time, rate))
            
            
file = pd.DataFrame(data)        

def cal_payment(data):
    

In [4]:
file.head(20)

,0,1,2,3,4,5,6,7,8,9
0,Aaron Edwards,2021-03-22,2021-03-28,2021-03-26,2021-03-26 22:00:00,2021-03-27 00:30:00,Friday,Saturday,2.5,1.35
1,Aaron Edwards,2021-03-22,2021-03-28,2021-03-26,2021-03-26 22:00:00,2021-03-27 02:00:00,Friday,Saturday,2.0,1.45
2,Aaron Edwards,2021-03-22,2021-03-28,2021-03-27,Friday,Saturday,2,1.7,NaN,NaN
3,Aaron Edwards,2021-03-22,2021-03-28,2021-03-27,2021-03-27 22:00:00,2021-03-28 07:00:00,Saturday,Sunday,2.0,1.70
4,Aaron Edwards,2021-03-22,2021-03-28,2021-03-28,Saturday,Sunday,7,2.2,NaN,NaN


In [5]:
# file.to_excel("1205.xlsx")